In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
!pip install gdown
import gdown
import numpy as np

# Define the Google Drive file ID
file_id = '1Rh8QGBZdxOzZW4rnqvtgmgPIzwI2JB3l'

# Construct the direct download URL
url = f'https://drive.google.com/uc?id={file_id}'

# Provide the output file path
output = 'wavelengths_400940-nm.npy'

# Download the file
gdown.download(url, output, quiet=False)

# Load the numpy array from the file
wavelengths = np.load(output)

# Use the wavelengths array as needed
# Example: Print the shape of the array
print(wavelengths.shape)

Downloading...
From: https://drive.google.com/uc?id=1Rh8QGBZdxOzZW4rnqvtgmgPIzwI2JB3l
To: /content/wavelengths_400940-nm.npy
100%|██████████| 4.46k/4.46k [00:00<00:00, 8.36MB/s]

(541,)


In [3]:
!pip install gdown
import gdown

file_id = '1GvYzw1VDbsEiwodPrJjdkEVGkP9iF0Wj'  # Replace with the actual file ID
output_file = 'omegga_egg_data_dataframe_v2.pkl'  # Specify the desired output file name

url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1GvYzw1VDbsEiwodPrJjdkEVGkP9iF0Wj
To: /content/omegga_egg_data_dataframe_v2.pkl
100%|██████████| 1.18G/1.18G [00:11<00:00, 107MB/s]


'omegga_egg_data_dataframe_v2.pkl'

In [ ]:
#Load the data

In [9]:
data_df = pd.read_pickle('omegga_egg_data_dataframe_v2.pkl')

In [8]:
wavelengths = np.load('wavelengths_400940-nm.npy')

In [ ]:
#explore the data


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [10]:
# Convert 'original_inc_seconds' column to hours
data_df['original_inc_hours'] = data_df['original_inc_seconds'] / 3600

In [11]:
# Perform feature engineering
features = []

In [12]:
for i in range(8):  # Iterate over the sub-measure positions
    for j in range(541):  # Iterate over the wavelengths
        feature_name = f'sub_measure_{i}_wavelength_{j}'
        feature_values = data_df['y_transmission'].apply(lambda x: x[i, j])
        data_df[feature_name] = feature_values
        features.append(feature_name)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-94db730c9d67>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[feature_name] = feature_values
<ipython-input-12-94db730c9d67>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[feature_name] = feature_values
<ipython-input-12-94db730c9d67>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) 

In [16]:
# Convert 'egg_number' column to integers
data_df['egg_number'] = data_df['egg_number'].astype(int)

# Prepare the features and target variables
X = data_df[features]
y = data_df['sex']  # Replace 'target_variable' with the correct column name for the target variable

# Initialize StratifiedKFold using 'egg_number' as the groups
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store ROC AUC scores
roc_auc_scores = []

# Perform cross-validation
for train_index, test_index in cv.split(X, y, groups=data_df['egg_number']):
    X_train, X_test = X.iloc[train_index].values, X.iloc[test_index].values
    y_train, y_test = y.iloc[train_index].values, y.iloc[test_index].values

    # Perform imputation to handle missing values
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    # Scale the data using a scaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)

    # Train the HistGradientBoostingClassifier
    model = HistGradientBoostingClassifier()
    model.fit(X_train_scaled, y_train)

    # Predict probabilities for the test set
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc_auc_scores.append(roc_auc)

# Print the average ROC AUC score across cross-validation folds
print("Average ROC AUC score:", np.mean(roc_auc_scores))


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:48: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow e

Average ROC AUC score: 0.7085895092345295


In [ ]:
#Average ROC AUC score: 0.7085895092345295


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [15]:
pip install numpy pandas scikit-learn


In [18]:
print(data_df.columns)


Index(['egg_number', 'sex', 'split_id', 'y_transmission',
       'original_inc_seconds', 'batch', 'sub_measure_0_wavelength_0',
       'sub_measure_0_wavelength_1', 'sub_measure_0_wavelength_2',
       'sub_measure_0_wavelength_3',
       ...
       'sub_measure_7_wavelength_531', 'sub_measure_7_wavelength_532',
       'sub_measure_7_wavelength_533', 'sub_measure_7_wavelength_534',
       'sub_measure_7_wavelength_535', 'sub_measure_7_wavelength_536',
       'sub_measure_7_wavelength_537', 'sub_measure_7_wavelength_538',
       'sub_measure_7_wavelength_539', 'sub_measure_7_wavelength_540'],
      dtype='object', length=4334)


In [19]:
print(data_df.head())


          egg_number sex  split_id  \
0  13_000000_000_000   m         3   
1  13_000000_000_002   f         3   
2  13_000000_000_003   f         3   
3  13_000000_000_004   m         3   
4  13_000000_001_000   m         2   

                                      y_transmission  original_inc_seconds  \
0  [[0.3618363437328931, 0.37204615865715585, 0.3...               49474.0   
1  [[0.3359087373606431, 0.3502913296932728, 0.41...               49521.0   
2  [[0.2568742257943702, 0.23089335209486356, 0.3...               49546.0   
3  [[0.26404445020597456, 0.09256002475830233, 0....               49572.0   
4  [[0.2625408496732011, 0.36669491521643377, 0.3...               49695.0   

   batch  sub_measure_0_wavelength_0  sub_measure_0_wavelength_1  \
0     13                    0.361836                    0.372046   
1     13                    0.335909                    0.350291   
2     13                    0.256874                    0.230893   
3     13                    0.

In [20]:
keyword = "target"  # Specify the keyword you are looking for
for column in data_df.columns:
    if keyword in column:
        print(column)


In [23]:
#(['egg_number', 'sex', 'split_id', 'y_transmission', 'original_inc_seconds', 'batch', ...], dtype='object', length=4334)


In [ ]:
import pandas as pd

# Assuming you have loaded the data into a Pandas DataFrame called 'data'
print(data_df.shape)  # Shape of the entire DataFrame


(34079, 6)


In [ ]:
import pandas as pd

# Check for extreme values
data_df.describe()

,split_id,original_inc_seconds,batch
count,34079.000000,34079.000000,34079.000000
mean,4.625194,334741.094139,14.399689
std,2.831950,170810.741085,1.114239
min,0.000000,39060.000000,13.000000
25%,2.000000,186800.500000,13.000000
50%,5.000000,334628.000000,14.000000
75%,7.000000,483464.500000,15.000000
max,9.000000,630528.000000,16.000000


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
#Preparing the features and target variable

In [ ]:
print(data_df.head())

          egg_number sex  split_id  \
0  13_000000_000_000   m         3   
1  13_000000_000_002   f         3   
2  13_000000_000_003   f         3   
3  13_000000_000_004   m         3   
4  13_000000_001_000   m         2   

                                      y_transmission  original_inc_seconds  \
0  [[0.3618363437328931, 0.37204615865715585, 0.3...               49474.0   
1  [[0.3359087373606431, 0.3502913296932728, 0.41...               49521.0   
2  [[0.2568742257943702, 0.23089335209486356, 0.3...               49546.0   
3  [[0.26404445020597456, 0.09256002475830233, 0....               49572.0   
4  [[0.2625408496732011, 0.36669491521643377, 0.3...               49695.0   

   batch  
0     13  
1     13  
2     13  
3     13  
4     13  


In [ ]:
print(data_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34079 entries, 0 to 642
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   egg_number            34079 non-null  object 
 1   sex                   34079 non-null  object 
 2   split_id              34079 non-null  int64  
 3   y_transmission        34079 non-null  object 
 4   original_inc_seconds  34079 non-null  float64
 5   batch                 34079 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.8+ MB
None


In [ ]:
column_names = data_df.columns
print(column_names)

Index(['egg_number', 'sex', 'split_id', 'y_transmission',
       'original_inc_seconds', 'batch'],
      dtype='object')


In [ ]:
# Check the class distribution

In [ ]:
print(data_df['sex'].value_counts())

m    17437
f    16642
Name: sex, dtype: int64


In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [17]:
#With this modification, the code should perform cross-validation correctly